In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
%load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [18]:
#Change the date to the previous day
query = """
WITH DATA AS (
SELECT DENSE_RANK() OVER (PARTITION BY TEAM_NAME ORDER BY GAME_DATE ASC) RK, TEAM_NAME, GAME_DATE, MATCHUP,WL, CASE WHEN WL='W' THEN 1 ELSE 0 END AS POINT, PLUS_MINUS FROM `perceptive-ivy-290216.nba_api_team_boxscore.team_box_96_onwards`
WHERE SEASON='2022-23'
-- AND TEAM_NAME='Cleveland Cavaliers'
ORDER BY TEAM_NAME,GAME_DATE
)

SELECT *,
SUM(POINT) OVER (PARTITION BY TEAM_NAME ORDER BY GAME_DATE ASC) WINS_ORDERED,
ROUND(SUM(POINT) OVER (PARTITION BY TEAM_NAME ORDER BY GAME_DATE ASC)/RK,4) WIN_PCT_ORDERED
FROM DATA
ORDER BY TEAM_NAME, GAME_DATE
"""
project_id = "perceptive-ivy-290216"
df_bq = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

In [19]:
df_bq.head()

,RK,TEAM_NAME,GAME_DATE,MATCHUP,WL,POINT,PLUS_MINUS,WINS_ORDERED,WIN_PCT_ORDERED
0,1,Atlanta Hawks,2022-10-19,ATL vs. HOU,W,1,10,1,1.0000
1,2,Atlanta Hawks,2022-10-21,ATL vs. ORL,W,1,10,2,1.0000
2,3,Atlanta Hawks,2022-10-23,ATL vs. CHA,L,0,-17,2,0.6667
3,4,Atlanta Hawks,2022-10-26,ATL @ DET,W,1,5,3,0.7500
4,5,Atlanta Hawks,2022-10-28,ATL @ DET,W,1,24,4,0.8000


In [20]:
px.line(df_bq, x="RK", y="WIN_PCT_ORDERED", color='TEAM_NAME',
        hover_data=['TEAM_NAME','GAME_DATE', 'MATCHUP', 'WL', 'PLUS_MINUS', 'WINS_ORDERED', 'WIN_PCT_ORDERED'], height=2000, width=1500,
        )

In [21]:
px.line(df_bq, x="RK", y="WINS_ORDERED", color='TEAM_NAME',
        hover_data=['TEAM_NAME','GAME_DATE', 'MATCHUP', 'WL', 'PLUS_MINUS', 'WINS_ORDERED', 'WIN_PCT_ORDERED'], height=1000, width=1500,
        )